In [155]:
import pandas as pd
import numpy as np

import time
import datetime

import os

from data_utils import BasicFunctions, PriceFunctions
from TechnicalAnalysis import TechnicalAnalysis 

In [2]:
bf = BasicFunctions()
pf = PriceFunctions()

coins = ['BTC', 'ETH', 'DASH', 'DOGE', 'LTC', 'STR', 'XMR', 'XRP']

dfs = {}
merged_dfs = {}

pd_Xtrains = {}
pd_ytrains = {}
pd_Xtests = {}
pd_ytests = {}

pd_Xtrains_nd = {}
pd_Xtests_nd = {}

means = {}
stds = {}

Xtrains = {}
ytrains = {}
Xtests = {}
ytests = {}


for coin in coins:
    dfs[coin] = pf.get_pandas(coin=coin, targetdays=24, absolute=True)

dfs = pf.to_same_starting(dfs)
dfs = pf.to_usd(dfs)

for coin in coins:
    ta = TechnicalAnalysis(dfs[coin], Timeframe=[24], period=[14], coin=coin)
    ta.merge_time(cache=True)

    merged_dfs[coin] = ta.get_dic()['24hour']
    

for coin in coins:
    pd_Xtrains[coin], pd_ytrains[coin], pd_Xtests[coin], pd_ytests[coin] = pf.split_traintest(merged_dfs[coin])

    pd_Xtrains_nd[coin] = pd_Xtrains[coin].drop('Date', axis=1)
    pd_Xtests_nd[coin] = pd_Xtests[coin].drop('Date', axis=1)

    means[coin], stds[coin], Xtrains[coin], ytrains[coin], Xtests[coin], ytests[coin] = pf.get_numpy(pd_Xtrains_nd[coin], pd_ytrains[coin], pd_Xtests_nd[coin], pd_ytests[coin])

Read from cache
Read from cache
Read from cache
Read from cache
Read from cache
Read from cache
Read from cache
Read from cache


In [123]:
def addIrregularFeatures(df_coin, irregular_data):
    '''
    Parameters:
    ___________
    
    df_coin (DataFrame):
    Dataframe of coin with date as the index.
    
    irregular_data(DataFrame):
    Dataframe with Date as column
    
    Returns:
    Dataframe the same size as df_coin with same date and features from irregular_data at the closest time - forward filled
    '''
    newDf = pd.DataFrame(columns=['Date'])
    newDf['Date'] = df_coin.index
    
    closestDf = pd.DataFrame(columns=['Date'])

    
    #replace with closest date
    for i in range(irregular_data['Date'].shape[0]):
        closestDf = closestDf.append({'Date': newDf.iloc[(newDf['Date'] - irregular_data['Date'].iloc[i]).abs().argsort()[0]]['Date']}, ignore_index=True)
    
    
    newTrends = irregular_data
    newTrends['Date'] = closestDf['Date']

    newTrends = newTrends.set_index('Date')
    newDf = newDf.set_index('Date')
    
    newTrends = newTrends[~newTrends.index.duplicated(keep='last')] #replace duplicates
    test = newDf.join(newTrends).ffill()
    
    return test #can add unit test if time

In [151]:
def add_trends(df, coinfull):
    '''
    Parameters:
    df (Dataframe):
    Dataframe containing coin price and all

    coinfull (string):
    Full name in small like bitcoin
    '''
    trend = pd.read_csv('data_utils\\trends_data\\{}.csv'.format(coinfull))
    trend = trend[::-1] #reverse
    trend = trend.reset_index(drop=True)
    
    trend['Date'] = trend['date'].apply(lambda x: int(time.mktime(datetime.datetime.strptime(x, "%Y-%M-%d").timetuple())))
    trend = trend.drop('date', axis=1)
    
    trend = trend.rename(columns={coinfull: 'Google Trend'})
    
    regFeatures = addIrregularFeatures(df, trend.drop('isPartial', axis=1))
    
    df = df.join(regFeatures)
    
    return df

In [152]:
add_trends(dfs['BTC'], 'bitcoin').head()

,Open,Close,High,Low,Volume,Percentage Change,Classification,Google Trend
Date,,,,,,,,
1439010000,278.46,278.31,278.72,277.37,709.882511,0.058615,0.0,2.0
1439013600,278.36,277.99,278.39,277.98,194.195152,0.059010,0.0,2.0
1439017200,277.98,278.07,278.07,277.96,54.156776,0.061295,0.0,2.0
1439020800,278.07,278.34,278.35,278.06,136.694550,0.063178,0.0,2.0
1439024400,278.35,278.34,278.35,278.33,94.217294,0.053241,0.0,2.0


In [149]:
def add_wikipedia(df, coinfull):
    '''
    Parameters:
    df (Dataframe):
    Dataframe containing coin price and all

    coinfull (string):
    Full name in small like bitcoin
    '''  
    
    wikiDf = pd.read_csv('data_utils\\wikipedia_data\\pageviews.csv')[['Date', coinfull]]
    wikiDf['Date'] = wikiDf['Date'].apply(lambda x: int(time.mktime(datetime.datetime.strptime(x, "%Y-%M-%d").timetuple())))
    wikiDf = wikiDf.rename(columns={coinfull: 'Wikipedia View'})
    
    regFeatures = addIrregularFeatures(df, wikiDf)
    
    df = df.join(regFeatures)
    
    return df

In [154]:
add_wikipedia(dfs['BTC'], 'bitcoin').head()

,Open,Close,High,Low,Volume,Percentage Change,Classification,Wikipedia View
Date,,,,,,,,
1439010000,278.46,278.31,278.72,277.37,709.882511,0.058615,0.0,7240.0
1439013600,278.36,277.99,278.39,277.98,194.195152,0.059010,0.0,7240.0
1439017200,277.98,278.07,278.07,277.96,54.156776,0.061295,0.0,7240.0
1439020800,278.07,278.34,278.35,278.06,136.694550,0.063178,0.0,7240.0
1439024400,278.35,278.34,278.35,278.33,94.217294,0.053241,0.0,7240.0


In [183]:
def add_blockchain(df, coinfull):
    
    '''
    Parameters:
    df (Dataframe):
    Dataframe containing coin price and all

    coinfull (string):
    Full name in small like bitcoin
    '''  
    
    files = os.listdir("data_utils\\blockchain_data\\bitcoin")

    dfBlock = pd.read_csv('data_utils\\blockchain_data\\bitcoin\\difficulty.csv', header=None)

    dfBlock.columns = ['Date', 'difficulty']
    
    for file in files:
        if file != 'difficulty.csv':
            tDf = pd.read_csv('data_utils\\blockchain_data\\bitcoin\\{}'.format(file), header=None)

            dfBlock[file[:-4]] = tDf[1]
            
    dfBlock['Date'] = dfBlock['Date'].apply(lambda x: int(time.mktime(datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").timetuple())))
    
    
    dfBlock = dfBlock.ffill()
    
    regFeatures = addIrregularFeatures(df, dfBlock)
    
    df = df.join(regFeatures)
    
    return df

In [185]:
add_blockchain(dfs['BTC'], 'bitcoin').head()

,Open,Close,High,Low,Volume,Percentage Change,Classification,difficulty,cost-per-transaction-percent,cost-per-transaction,estimated-transaction-volume-usd,estimated-transaction-volume,hash-rate,miners-revenue,n-transactions,output-volume,transaction-fees
Date,,,,,,,,,,,,,,,,,
1439010000,278.46,278.31,278.72,277.37,709.882511,0.058615,0.0,5.269984e+10,1.58203,10.20423,62843015.0,236325.285714,411296.56997,1065525.72,104420.0,1.121561e+06,23.042149
1439013600,278.36,277.99,278.39,277.98,194.195152,0.059010,0.0,5.269984e+10,1.58203,10.20423,62843015.0,236325.285714,411296.56997,1065525.72,104420.0,1.121561e+06,23.042149
1439017200,277.98,278.07,278.07,277.96,54.156776,0.061295,0.0,5.269984e+10,1.58203,10.20423,62843015.0,236325.285714,411296.56997,1065525.72,104420.0,1.121561e+06,23.042149
1439020800,278.07,278.34,278.35,278.06,136.694550,0.063178,0.0,5.269984e+10,1.58203,10.20423,62843015.0,236325.285714,411296.56997,1065525.72,104420.0,1.121561e+06,23.042149
1439024400,278.35,278.34,278.35,278.33,94.217294,0.053241,0.0,5.269984e+10,1.58203,10.20423,62843015.0,236325.285714,411296.56997,1065525.72,104420.0,1.121561e+06,23.042149
